In [ ]:
import os 
import cv2
import numpy
import typing

In [ ]:
TRAIN_VIDEOS_PATH = "experiments/current_experiment/data/videos/"

# Video Processing methods

In [ ]:
def load_videos(self):
    paths = []
    for file_video in os.listdir(TRAIN_VIDEOS_PATH):
        full_path = os.path.join(TRAIN_VIDEOS_PATH, file_video)
        paths.append(full_path)
    return paths

def split_videos_in_sets(video_urls: numpy.ndarray, test_size_perc: float):
    """
    Function splits video urls array into 
    training and testing sets, according 
    to the given proportion of 'test set'.

    Args:
        - video_urls: list - list of video urls
    """
    if test_size >= 1: raise ValueError('test size cannot be 1 or 0 or any other value, greater than 1')
    test_idxs = numpy.random.randint(
        low=0, 
        high=len(video_urls)-1, 
        size=len(video_urls) * test_size_perc
    )
    train_video_idxs = numpy.arange(len(video_urls))[~random_test_idxs]
    return video_urls
    

def extract_quality_frames(video_url: str, number_of_imgs: int):
    """
    Function for extracting quality frames
    from given video file. Randomly generates 
    index frames to extract from the file within 
    the range and then extracts it.
    Args:
        - number_of_imgs - number of images to extract
        from video
    Returns:
        - list of image frames
    """
    output_frames = []
    video = cv2.VideoCapture()
    opened = video.open(filename=video_url)
    MAX_FRAMES = int(video.get(propId=cv2.CAP_PROP_FRAME_COUNT))

    if not opened: 
        raise RuntimeError("Failed to open video, file may be invalid")

    random_idxs = numpy.random.randint(
        low=0, 
        high=MAX_FRAMES-1, 
        size=number_of_imgs
    )

    curr_idx = 0

    while len(output_frames) != number_of_imgs and curr_idx < (MAX_FRAMES-1):
        extracted, curr_frame = video
        if not extracted: raise RuntimeError("Failed to extract video frame, file may be damaged")
        if curr_idx == random_idxs[0]:
            output_frames.append(curr_frame)
            random_idxs.pop(0)
        curr_idx = curr_idx + 1
    video.close()